# On-Disk Transductive Learning: Structure-Centric Batching

Train on massive graphs with **structure-complete mini-batches** and constant memory!

**What you'll learn:**
- 🎯 Structure-centric sampling (sample structures → gather nodes)
- ✅ 100% structure completeness by construction
- 📊 Which topological structures benefit
- 🔧 Integration with TopoBench pipeline

**📚 Prerequisites:** Complete `tutorial_ondisk_transductive_intro.ipynb` first!  
That tutorial covers: transductive learning basics, node samplers, cluster-aware sampling, structure loss problem, and on-disk indexing fundamentals.

**⏱️ Time:** 15-20 minutes

## Why Structure-Centric Batching?

**Paradigm shift from Tutorial 1:** Instead of sampling nodes, we sample **structures first**!

**Recap:** In Tutorial 1, we learned that traditional node-based sampling loses 20-40% of structures at cluster boundaries.

**This approach:**
- **Sample structures first** (e.g., triangles by ID)
- **Gather all nodes** needed for those structures  
- **Result:** All sampled structures are **100% complete** in the batch

**When to use:**
- Need **guaranteed structure completeness**
- Using enumerable structures (cliques, cycles)
- Willing to change from node-based sampling

## Which Topological Structures Benefit?

### ✅ Benefits from Structure-Centric:
- **SimplicialCliqueLifting**: Cliques/triangles enumerated from graph
- **CellCycleLifting**: Cycles discovered in graph
- **Any enumerable structure** from the graph topology

### ❌ Does NOT benefit:
- **HypergraphKHopLifting**: Neighborhoods generated dynamically per node
- **KernelLifting**: Structures computed from kernels, not enumerated
- **Feature-based** transforms that don't rely on graph structure enumeration

**Why?** Structure-centric requires **pre-enumerable structures** that can be indexed. Dynamic/generated structures don't fit this paradigm.

### Table of Contents

1. [Setup and Data Loading](#setup)
2. [Building the On-Disk Index](#index)
3. [Structure-Centric Dataloader](#dataloader)
4. [Training with TBModel](#training)
5. [Memory Efficiency](#memory)
6. [Do Liftings Need Modification?](#liftings)

<a id='setup'></a>
## 1. Setup and Data Loading

Following TopoBench conventions, we define a dataset class and loader.

In [ ]:
# Imports
import networkx as nx
import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.io import fs
import lightning as pl
from omegaconf import OmegaConf, DictConfig

# TopoBench imports
from topobench.data.loaders.base import AbstractLoader
from topobench.data.preprocessor import OnDiskTransductivePreprocessor
from topobench.dataloader import TBDataloader
from topobench.model import TBModel
from topobench.nn.backbones.simplicial import SCCNNCustom
from topobench.nn.readouts.mlp_readout import MLPReadout
from topobench.loss import TBLoss
from topobench.optimizer import TBOptimizer

print("✓ Imports complete")

### Define Dataset Class (TopoBench Style)

**Important: Why `InMemoryDataset` is fine here**

You might wonder: "If we're doing on-disk processing, why use `InMemoryDataset`?"

**Answer:** We keep the **graph** in memory, but **structures** on disk!

- **Graph data** (nodes, edges, features): ~50-200 MB for 100K nodes → **fits in RAM** ✅
- **Topological structures** (triangles, etc.): 1-10 GB → **stored on disk** ✅

**Why this works:**
1. The base graph must be in memory anyway (for subgraph extraction)
2. The **bottleneck** is structures, not the graph itself
3. Our on-disk index handles the structures (SQLite)
4. Result: Constant memory training on large graphs!

**Would on-demand graph loading help?** No, because:
- We need full graph in memory to extract subgraphs
- Preprocessor needs full graph to query edges
- Graph itself is not the memory problem
- Structures are the problem (and we solve that!)

**Bottom line:** `InMemoryDataset` for graph + on-disk index for structures = perfect combo!

### Define Dataset Class (TopoBench Style)

In [ ]:
class MyLargeTransductiveDataset(InMemoryDataset):
    """Large single graph for transductive learning."""
    
    def __init__(self, root, name, parameters: DictConfig):
        self.name = name
        self.parameters = parameters
        super().__init__(root)
        
        out = fs.torch_load(self.processed_paths[0])
        if len(out) == 4:
            data, self.slices, self.sizes, data_cls = out
            self.data = data_cls.from_dict(data) if isinstance(data, dict) else data
        else:
            data, self.slices, self.sizes = out
            self.data = data
    
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return "data.pt"
    
    def download(self):
        pass
    
    def process(self):
        """Generate large graph with community structure."""
        # Create graph (example: Watts-Strogatz)
        G = nx.watts_strogatz_graph(
            n=self.parameters.num_nodes,
            k=self.parameters.degree,
            p=0.3,
            seed=42
        )
        
        # Convert to PyG Data
        edges = list(G.edges())
        edge_index = torch.tensor(edges, dtype=torch.long).t()
        edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)
        
        n = self.parameters.num_nodes
        x = torch.randn(n, self.parameters.num_features)
        y = torch.randint(0, self.parameters.num_classes, (n,))
        
        # Transductive splits
        train_mask = torch.zeros(n, dtype=torch.bool)
        val_mask = torch.zeros(n, dtype=torch.bool)
        test_mask = torch.zeros(n, dtype=torch.bool)
        
        train_mask[:int(0.6 * n)] = True
        val_mask[int(0.6 * n):int(0.8 * n)] = True
        test_mask[int(0.8 * n):] = True
        
        data = Data(
            x=x, edge_index=edge_index, y=y, num_nodes=n,
            train_mask=train_mask, val_mask=val_mask, test_mask=test_mask
        )
        
        self.data, self.slices = self.collate([data])
        fs.torch_save(
            (self._data.to_dict(), self.slices, {}, self._data.__class__),
            self.processed_paths[0]
        )

print("✓ Dataset class defined")

### Define Loader Class (TopoBench Style)

In [ ]:
class MyLargeTransductiveLoader(AbstractLoader):
    """Loader for large transductive datasets."""
    
    def __init__(self, parameters: DictConfig):
        super().__init__(parameters)
    
    def load_dataset(self):
        return MyLargeTransductiveDataset(
            str(self.root_data_dir),
            self.parameters.data_name,
            self.parameters
        )

print("✓ Loader class defined")

In [ ]:
# Configuration
config = OmegaConf.create({
    "data_dir": "./data/",
    "data_name": "MyLargeGraph",
    "num_nodes": 10000,
    "degree": 20,
    "num_features": 32,
    "num_classes": 4
})

# Load dataset
loader = MyLargeTransductiveLoader(config)
dataset, _ = loader.load()
graph_data = dataset[0]

print(f"\n✓ Graph loaded: {graph_data.num_nodes:,} nodes")
print(f"  Edges: {graph_data.edge_index.size(1):,}")
print(f"  Train nodes: {graph_data.train_mask.sum().item():,}")

<a id='index'></a>
## 2. Building the On-Disk Index

**Key Innovation:** Index structures once to SQLite database on disk.

**What happens:**
1. Enumerate structures (e.g., triangles) from graph
2. Stream results to SQLite database (disk-based)
3. Close index file
4. Database persists for future use

**Memory:** Constant during indexing (streaming enumeration)

In [ ]:
# Configure transform (applied at batch-time)
transforms_config = OmegaConf.create({
    "clique_lifting": {
        "transform_type": "lifting",
        "transform_name": "SimplicialCliqueLifting",
        "complex_dim": 2  # Triangles
    }
})

# Create preprocessor (no need to build_index manually - it's automatic!)
preprocessor = OnDiskTransductivePreprocessor(
    graph_data=graph_data,
    data_dir="./index/structure_centric_demo",
    transforms_config=transforms_config,
    max_structure_size=3  # Triangles = 3 nodes
)

print("✓ Preprocessor created")

<a id='dataloader'></a>
## 3. Load Dataset Splits (TopoBench Style)

**High-level API:** Exactly like inductive learning!

**What happens under the hood:**
1. Builds structure index (if not exists) → saved to disk
2. Creates train/val/test datasets based on masks
3. Each dataset wraps a loader for structure-centric sampling

**Result:** Train, val, test datasets ready for TBDataloader!

In [ ]:
# Create split configuration (like inductive learning!)
split_config = OmegaConf.create({
    "strategy": "structure_centric",
    "structures_per_batch": 500,  # Target number of structures
    "node_budget": 2000,          # Max nodes (controls memory)
})

print("🗄️ Loading dataset splits (builds index if needed)...\n")

# Load splits - EXACTLY like inductive learning!
train, val, test = preprocessor.load_dataset_splits(split_config)

print(f"\n✓ Dataset splits loaded!")
print(f"  Train: {len(train)} batches")
print(f"  Val: {len(val)} batches")
print(f"  Test: {len(test)} batches")
print(f"  Strategy: Structure-centric batching")

# Inspect a sample batch
sample_batch = next(iter(train))
print(f"\n📦 Sample training batch:")
print(f"  Nodes: {sample_batch.num_nodes}")
print(f"  Edges: {sample_batch.edge_index.size(1)}")
print(f"  Structures: {sample_batch.num_structures}")
print(f"  Has precomputed structures: {hasattr(sample_batch, 'precomputed_structures')}")

### Create Datamodule (TopoBench Style)

**Exactly like inductive learning:** Use TBDataloader!

In [ ]:
# Create datamodule - EXACTLY like inductive learning!
datamodule = TBDataloader(
    dataset_train=train,
    dataset_val=val,
    dataset_test=test,
    batch_size=1,  # Already batched by dataset
    num_workers=0
)

print("✓ Datamodule created (TopoBench style)")
print("  This is IDENTICAL to inductive learning!")
print("  Same API, same workflow, just different sampling strategy")

In [ ]:
<a id='training'></a>
## 4. Training with TBModel

**Integration:** Works seamlessly with TopoBench's TBModel!

**Training flow:**
1. Loader yields batch with complete structures
2. Transform (SimplicialCliqueLifting) applied at batch-time
3. Model processes batch
4. Standard training loop

# Define model
HIDDEN_DIM = 64
OUT_CHANNELS = 4
IN_CHANNELS = 32

model = TBModel(
    backbone=SCCNNCustom(
        in_channels_all=(IN_CHANNELS, HIDDEN_DIM, HIDDEN_DIM),
        hidden_channels_all=(HIDDEN_DIM, HIDDEN_DIM, HIDDEN_DIM),
        conv_order=1,
        sc_order=2,
        n_layers=2
    ),
    readout=MLPReadout(
        HIDDEN_DIM, OUT_CHANNELS, task_level="node"
    ),
    loss=TBLoss(
        dataset_loss={"task": "classification", "loss_type": "cross_entropy"}
    ),
    optimizer=TBOptimizer(
        optimizer_id="Adam", parameters={"lr": 0.01}
    )
)

print("✓ TBModel created")
print("  Backbone: SCCNN (Simplicial Convolutional Network)")
print("  Task: Node classification")

In [ ]:
# Train - EXACTLY like inductive learning!
trainer = pl.Trainer(
    max_epochs=5,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True
)

print("\n🚀 Training with structure-centric batching...\n")

trainer.fit(model, datamodule)

print("\n✅ Training complete!")
print(f"  Trained on {graph_data.train_mask.sum().item():,} train nodes")
print(f"  Validated on {graph_data.val_mask.sum().item():,} val nodes")
print(f"  Memory per batch: ~200-300 MB")
print(f"  All batches had complete structures!")

<a id='memory'></a>
## 5. Memory Efficiency

**Key Advantage:** Memory scales with batch size, not graph size!

In [ ]:
# Estimate memory
def estimate_batch_memory(batch):
    """Estimate memory usage in MB."""
    total_bytes = 0
    if hasattr(batch, 'x'):
        total_bytes += batch.x.element_size() * batch.x.nelement()
    if hasattr(batch, 'edge_index'):
        total_bytes += batch.edge_index.element_size() * batch.edge_index.nelement()
    if hasattr(batch, 'y'):
        total_bytes += batch.y.element_size() * batch.y.nelement()
    return total_bytes / (1024 * 1024)

# Measure across batches
memories = []
for i, batch in enumerate(datamodule.train_dataloader()):
    if i >= 20:
        break
    memories.append(estimate_batch_memory(batch))

print("\n💾 Memory Analysis:")
print(f"  Average batch: {sum(memories)/len(memories):.1f} MB")
print(f"  Min batch: {min(memories):.1f} MB")
print(f"  Max batch: {max(memories):.1f} MB")
print(f"\n  Full graph (if in-memory): ~{graph_data.num_nodes * 32 * 4 / 1024 / 1024:.0f} MB")
print(f"  Memory savings: {(graph_data.num_nodes * 32 * 4 / 1024 / 1024) / (sum(memories)/len(memories)):.0f}x")
print("\n✅ Memory scales with batch size, not graph size!")

<a id='liftings'></a>
## 6. Do Liftings Need Modification?

**Short Answer:** No! Existing liftings work as-is.

**How it works:**
- **Index building:** Preprocessor enumerates structures using the lifting's logic
- **Batch-time:** Lifting is applied to mini-batch (same as always)
- **Difference:** Structures are queried from index instead of computed fresh

**Example with SimplicialCliqueLifting:**

In [ ]:
# Cleanup
preprocessor.close()
print("✓ Tutorial complete!")